In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from googleapiclient.discovery import build

# ✅ 1. 환경변수 로드
load_dotenv(dotenv_path=r"c:/Users/NOTE2/조원C&I/개인biz/phytho code/youtube analytics/.env")
API_KEY = os.getenv('YOUTUBE_API_KEY')
youtube = build('youtube', 'v3', developerKey=API_KEY)

# ✅ 2. 키워드별 영상 ID 수집
def get_video_ids_by_keyword(keyword, max_results, start_date, end_date):
    request = youtube.search().list(
        q=keyword,
        part='id',
        type='video',
        maxResults=max_results,
        order='date',
        publishedAfter=start_date,
        publishedBefore=end_date
    )
    response = request.execute()
    return [item['id']['videoId'] for item in response['items']]

# ✅ 3. 영상 상세 정보 수집
def get_video_details(video_ids, keyword):
    video_data = []
    for i in range(0, len(video_ids), 50):
        ids = video_ids[i:i+50]
        request = youtube.videos().list(
            part='snippet,statistics,contentDetails',
            id=','.join(ids)
        )
        response = request.execute()
        for item in response['items']:
            stats = item.get('statistics', {})
            snippet = item['snippet']
            video_data.append({
                'keyword': keyword,
                'title': snippet['title'],
                'channel': snippet['channelTitle'],
                'pub_date': snippet['publishedAt'],
                'counts': stats.get('viewCount', '0'),
                'likes_count': stats.get('likeCount', '0'),
                'comment_count': stats.get('commentCount', '0'),
                'url': f"https://www.youtube.com/watch?v={item['id']}"
            })
    return video_data

# ✅ 4. 실행 및 저장
if __name__ == '__main__':
    keywords = ["김문수 후보 등록", "이재명 후보 등록", "대선 후보 등록"]
    start_date = "2025-05-10T00:00:00Z"
    end_date = "2025-05-11T23:59:59Z"
    max_results = 30

    all_results = []

    for keyword in keywords:
        print(f"\n🔍 '{keyword}' 영상 수집 중...")
        video_ids = get_video_ids_by_keyword(keyword, max_results, start_date, end_date)
        video_details = get_video_details(video_ids, keyword)
        print(f"✅ {len(video_details)}건 수집됨")
        all_results.extend(video_details)

    df = pd.DataFrame(all_results)

    # ✅ 미리보기 출력
    print("\n📋 상위 5개 미리보기:")
    print(df[['keyword', 'title', 'channel', 'pub_date', 'url']].head(5))

    # ✅ 날짜 및 키워드 요약으로 파일명 생성
    date_tag = start_date[2:10].replace('-', '')  # '250510'
    keyword_tag = '_'.join([kw.split()[0] for kw in keywords])  # '김문수_이재명_대선'
    output_file = f"youtube_metadata_{date_tag}_{keyword_tag}.csv"

    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"\n✅ 전체 {len(df)}건 저장 완료 → {output_file}")
